# Introduction

This notebook allows you to apply a simplified version of the ANTARESS workflow to a dataset obtained during an exoplanet transit, for the following purposes:
- characterizing systematic trends in out-of-transit stellar line properties 
- measuring the systemic velocity at the epoch of the transit

The notebook takes as input a time-series of disk-integrated CCFs, after they have been processed with the [reduction notebook](https://gitlab.unige.ch/spice_dune/antaress/-/blob/main/Notebooks/ANTARESS_nbook_sp_reduc.ipynb)). The default version is set up to analyze the echelle spectra obtained with ESPRESSO during the transit of TOI-421c, cleaned and converted into CCFs with the reduction notebook. Otherwise adjust settings to retrieve your own cleaned dataset, setting `working_path`, `star_name`, and `pl_name` to the same values that you used to [initialize](https://gitlab.unige.ch/spice_dune/antaress/-/blob/main/Notebooks/ANTARESS_nbook_setup.ipynb) your system.

To exploit the workflow to its full capabilities, run its executable with the [configuration files](https://obswww.unige.ch/~bourriev/antaress/doc/html/installation.html).

In [4]:
import ANTARESS_nbook_bground
input_nbook = {
    'working_path' : '/Users/bourrier/Travaux/ANTARESS/Working_dir/',
    'star_name' : 'TOI421',
    'pl_name'   : 'TOI421c'
}
input_nbook = ANTARESS_nbook_bground.load_nbook(input_nbook, 'trends')

TypeError: init() takes 0 positional arguments but 1 was given

# CCF analysis

Trends are searched for in the properties of the disk-integrated stellar line.
The first step thus consists in fitting CCFs with a Gaussian model to derive their full width at half maxima (FWHM), contrast, and rv position. 

You can define the radial velocity range for the fit and continuum. 
- `cont_range` : define regions used for the continuum, which can be multiple ranges
- `fit_range` (list, float): define range used for fit, can be multiple ranges and will then have the same format as `cont_range`


Additional model and fit options are available through the configuration file. The full workflow will also allow you to correct for putative line trends.

**Fit disk-integrated CCF profiles**

The first step in fitting the CCF profiles will be to 



**FIT Model parameters**

To fit the CCFs, we use a Gaussian model using $\chi^2$-minimization. The guess parameters will be used to initialise the starting values of the fitting routine. Choose guess values for RV and FWHM in km/s and the contrast. To plot the Gaussian fit set `plot_CCF` to true, the properties for RV, FWHM, and contrast will be plotted if `plot=True`.


In [ ]:
# Ranges
input_nbook['par'].update({
    'cont_range' : [[-100., -25.], [25., 100.]],
    'fit_range'  : [[-100, 100]]
})


input_nbook['par']['guess'({
    'rv'       : 0.,
    'FWHM'     : 6.,
    'ctrst'    : 0.7,
})

# Set to True to run the fit, and to False to retrieve its results
input_nbook['par']['calc_fit'] = True  

ANTARESS_nbook_bground.ana_prof(input_nbook,'DI')

**Plotting disk-integrated CCFs**

Run this cell to plot the disk-integrated CCF series with their best fits. Uncomment `y_range` to adjust the flux range of the CCFs (vertical axis).

In [ ]:
input_nbook['par']['y_range'] = [2.1e5,3.6e5] 
ANTARESS_nbook_bground.plot_prof(input_nbook,'DI_prof')

<a id='fit_prop'></a>
# Trend characterisation

 will be analysed to see if they correlate with either time (phase) or SNR. For more options, use the configuration file.

Here, we investigate the data and the model parameters to identify any trends seen in the model parameters as either a function of time (phase) or SNR. The goal will be to find the model and its coefficients that best fit the data. No correction will be applied if a linear fit shows the best agreement with the data. The most common models to characterise the data are first or second-degree polynomials; if you need a more complex model to fit the data, e.g., a sinus model, you can use the full ANTARESS workflow using [configuration files](https://obswww.unige.ch/~bourriev/antaress/doc/html/installation.html). In this notebook, we only work with polynomials.

- `pol_deg` (integer): degree of polynomial to fit
- `x_var` (string): phase or SNR, choose what dependence to plot against for the properties (RV, FWHM, and ctrst).

Run the cell below to perform the fit for (RV, FWHM, and ctrst) as a function of `'phase'` or `'snr'`. Try different polynomial degrees. The Bayesian information criterion (BIC) will be printed for each fit to compare the different models.

Identify any trends usinc the cell [Trend characterisation](#fit_prop), and find the best model representing the data. Once the best models for the parameters RV, FWHM, and contrast have been found using the Bayesian Information Criterion, insert the coefficient for each model below. If multiple properties are being corrected, add them to the list as shown in the example below.


In [ ]:
input_nbook['DI_trend'].update({
    'pol_deg' : 0.,
    'x_var'   : 'phase',
})

ANTARESS_nbook_bground.fit_prop(input_nbook, plot=False)

input_nbook['settings']['plot_dic']['porp_DI'] = 'png'

**Systemic velocity**

**If it is the first time you run the analysis, leave it as** `gamma = 0`. When you run ANTARESS the first time, you will measure the systemic velocity, which will be printed in the output, later set gamma to the measured value.

 This step is used to measure the systemic rv, to check that your Keplerian model matches the data, and to search for systematic trends in the line properties. 

In [ ]:
input_nbook['par']['gamma'] = 0.
ANTARESS_nbook_bground.set_sysvel(input_nbook)

**Plotting disk-integrated line properties**

Run this cell to plot the time-series of properties derived from the fits to disk-integrated CCFs. 

In [ ]:
ANTARESS_nbook_bground.plot_prop(input_nbook,'DI')

# Running ANTARESS

Run this cell to run the ANTARESS workflow.

In [ ]:
from antaress.ANTARESS_launch.ANTARESS_launcher import ANTARESS_launcher
ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

# Plot display

In [ ]:
from IPython.display import Image,display
from IPython.core.display import HTML 

Run the cells below to show saved plots. Plots can be found in `working_path/targetName_plots/`. For the fits to the CCF go to `working_path/target/targetPL_Plots/DI_data/instrument_night_Indiv/Data/`

**Disk-integrated CCF series**

This plot shows the series of disk-integrated CCFs (blue profiles) with their best-fit Gaussian models (dashed black profiles), from which are derived the properties showed in the next plot.

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_data/'+input_nbook['par']['instrument']+'_'+input_nbook['par']['night']+'_Indiv/Data/CCF/phase.gif')

**Disk-integrated line properties**

The three plots below show the time-series of RVs, contrast, and FWHM derived from the fits to the disk-integrated CCFs. Transit contacts are shown as dashed black lines, and in-transit exposures are flagged with empty symbols.

Outside of the transit, RVs should trace the Keplerian motion of your star with respect to the solar system barycenter (the Keplerian model defined by your inputs for the star and planets is plotted as a solid blue curve). If your star is quiet and measurements are dominated by white noise, RV measurements should be evenly spread around the Keplerian model, while contrast and FWHM measurements should be evenly spread around their mean out-of-transit value (dashed blue lines). You can assess this by looking at the ratio between the dispersion and mean error over out-of-transit measurements, reported on each plot, which should be close to unity. If that is not the case, you can use the workflow to detrend the data.  

Inside of the transit the planet occults local regions of the stellar surface, distorting the disk-integrated stellar lines and inducing anomalous deviations in their derived properties. Traditionally, measurements of the sky-projected spin-orbit angle in planetary systems have been performed by analyzing this anomalous signal in RV - but the anomaly may not be detectable with faint, slow-rotating stars and small planets. With ANTARESS, you will be able to analyze directly the planet-occulted stellar profiles to get a higher precision on the transit chord and orbital architecture of the system.

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_prop/rv_phase.png')

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_prop/rv_res_phase.png')

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_prop/ctrst_phase.png')

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_prop/FWHM_phase.png')